# [SQLAlchemy Tutorial](https://www.youtube.com/watch?v=woKYyhLCcnU)

# Engine

In [125]:
from sqlalchemy import create_engine
engine = create_engine("sqlite:///core_tutorial.sqlite3", echo=True)
engine

Engine(sqlite:///core_tutorial.sqlite3)

> Methods  

<br>  

>> #1  

    result = engine.execute("write the sql query directly here")
    result.fetchall()

>> #2  

    conn = engine.connect()
    result = conn.execute("some statement")
    result.fetchall()
    conn.close()

>> #3  

    conn = engine.connect()
    transaction = conn.begin()
    conn.execute("statement1")
    conn.execute("statement2")
    transaction.commit()
    conn.close()

> Note: Directly sends string to the DBAPI. So, we need to take care of the syntax specificity of each database platform (different for sqlite3, postgresql, mysql etc)

# Table, Metadata, Reflection, DDL

In [126]:
from sqlalchemy import MetaData
from sqlalchemy import Table, Column
from sqlalchemy import Integer, String

metadata = MetaData()
user_table = Table('user', metadata,
                Column('id', Integer, primary_key=True),
                Column('name', String),
                Column('fullname', String)
                )

> user_table created above is a python object which represents the table named 'user'.  
> 
> This table object holds 3 column objects named 'id', 'name', 'fullname'.  

### Attributes of the table object

In [127]:
print(user_table.columns)
print(user_table.columns.keys())
user_table.columns

ImmutableColumnCollection(user.id, user.name, user.fullname)
['id', 'name', 'fullname']


In [128]:
print(user_table.columns.name)
user_table.columns.name

user.name


Column('name', String(), table=<user>)

In [129]:
print(user_table.columns.id)
user_table.columns.id

user.id


Column('id', Integer(), table=<user>, primary_key=True, nullable=False)

In [130]:
print(user_table.columns.fullname)
user_table.columns.fullname

user.fullname


Column('fullname', String(), table=<user>)

In [131]:
print(user_table.c)
user_table.c

ImmutableColumnCollection(user.id, user.name, user.fullname)


In [132]:
user_table.c.name.type

String()

In [133]:
user_table.c.id.type

Integer()

In [134]:
print(user_table.c.name.type)
print(user_table.c.id.type)

VARCHAR
INTEGER


In [135]:
user_table.primary_key

PrimaryKeyConstraint(Column('id', Integer(), table=<user>, primary_key=True, nullable=False))

In [136]:
print(user_table.select())

SELECT "user".id, "user".name, "user".fullname 
FROM "user"


### Use Table and Metadata objects to generate a schema in a database

In [137]:
from sqlalchemy import create_engine
engine = create_engine("sqlite:///core_tutorial.sqlite3", echo=True)
metadata.create_all(engine)

2022-02-01 01:51:24,424 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-02-01 01:51:24,425 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user")
2022-02-01 01:51:24,426 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-01 01:51:24,428 INFO sqlalchemy.engine.Engine COMMIT


### Many different datatypes are supported

In [138]:
fancy_table.drop(engine, checkfirst=True)
# engine.execute("drop table if exists fancy")

2022-02-01 01:51:24,495 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-02-01 01:51:24,496 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("fancy")
2022-02-01 01:51:24,496 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-01 01:51:24,497 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("fancy")
2022-02-01 01:51:24,498 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-01 01:51:24,499 INFO sqlalchemy.engine.Engine COMMIT


In [139]:
from sqlalchemy import String, Numeric, DateTime, Enum

fancy_table = Table('fancy', metadata,
                Column('key', String(50), primary_key=True),
                Column('timestamp', DateTime),
                Column('amount', Numeric(10, 2)),
                Column('type', Enum('a', 'b', 'c'))
            )
fancy_table.create(engine)

2022-02-01 01:51:24,534 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-02-01 01:51:24,537 INFO sqlalchemy.engine.Engine 
CREATE TABLE fancy (
	"key" VARCHAR(50) NOT NULL, 
	timestamp DATETIME, 
	amount NUMERIC(10, 2), 
	type VARCHAR(1), 
	PRIMARY KEY ("key")
)


2022-02-01 01:51:24,538 INFO sqlalchemy.engine.Engine [no key 0.00114s] ()
2022-02-01 01:51:24,545 INFO sqlalchemy.engine.Engine COMMIT


### Foreign key constraint

In [140]:
address_table.drop(engine, checkfirst=True)
# engine.execute("drop table if exists address")

2022-02-01 01:51:24,618 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-02-01 01:51:24,620 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-02-01 01:51:24,621 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-01 01:51:24,624 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2022-02-01 01:51:24,625 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-01 01:51:24,628 INFO sqlalchemy.engine.Engine COMMIT


In [141]:
from sqlalchemy import ForeignKey
address_table = Table('address', metadata,
                    Column('id', Integer, primary_key=True),
                    Column('email_address', String(100), nullable=False),
                    Column('user_id', Integer, ForeignKey('user.id'))
                )
address_table.create(engine)

2022-02-01 01:51:24,684 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-02-01 01:51:24,686 INFO sqlalchemy.engine.Engine 
CREATE TABLE address (
	id INTEGER NOT NULL, 
	email_address VARCHAR(100) NOT NULL, 
	user_id INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(user_id) REFERENCES user (id)
)


2022-02-01 01:51:24,688 INFO sqlalchemy.engine.Engine [no key 0.00217s] ()
2022-02-01 01:51:24,694 INFO sqlalchemy.engine.Engine COMMIT


> For composite foreign key constraints watch SQLAlchemy tutorial at [51:10](https://www.youtube.com/watch?v=woKYyhLCcnU)

In [142]:
network_table.drop(engine, checkfirst=True)
# engine.execute("drop table if exists network")

2022-02-01 01:51:24,719 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-02-01 01:51:24,720 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("network")
2022-02-01 01:51:24,722 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-01 01:51:24,723 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("network")
2022-02-01 01:51:24,725 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-01 01:51:24,726 INFO sqlalchemy.engine.Engine COMMIT


In [143]:
from sqlalchemy import String, Integer, DateTime

network_table = Table('network', metadata,
                    Column('network_id', Integer, primary_key=True),
                    Column('name', String(100), nullable=False),
                    Column('created_at', DateTime, nullable=False),
                    Column('owner_id', Integer, ForeignKey('user.id'))
                )
metadata.create_all(engine)

2022-02-01 01:51:24,763 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-02-01 01:51:24,764 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user")
2022-02-01 01:51:24,765 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-01 01:51:24,767 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("fancy")
2022-02-01 01:51:24,768 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-01 01:51:24,769 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-02-01 01:51:24,771 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-01 01:51:24,773 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("network")
2022-02-01 01:51:24,774 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-01 01:51:24,775 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("network")
2022-02-01 01:51:24,777 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-01 01:51:24,779 INFO sqlalchemy.engine.Engine 
CREATE TABLE network (
	network_id INTEGER NOT NULL, 
	name VARCHAR(100) NOT NULL, 
	created_at DATETIME

# Reflection

> Refers to loading Table objects based on reading from an existing database.  
> Note: We are creating new metadata object called metadata2 (Because 'metadata' object created above already has a 'user' table inside it).

In [144]:
metadata2 = MetaData()
user_reflected = Table('user', metadata2, autoload=True, autoload_with=engine)

2022-02-01 01:51:24,834 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("user")
2022-02-01 01:51:24,834 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-01 01:51:24,837 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-02-01 01:51:24,837 INFO sqlalchemy.engine.Engine [raw sql] ('user',)
2022-02-01 01:51:24,838 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("user")
2022-02-01 01:51:24,839 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-01 01:51:24,840 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("user")
2022-02-01 01:51:24,841 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-01 01:51:24,842 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-02-01 01:51:24,842 INFO sqlalchemy.engine.Engine [raw sql] ('user',)
2022-02-01 01:51:24,843

In [145]:
user_reflected

Table('user', MetaData(), Column('id', INTEGER(), table=<user>, primary_key=True, nullable=False), Column('name', VARCHAR(), table=<user>), Column('fullname', VARCHAR(), table=<user>), schema=None)

In [146]:
network_reflected = Table('network', metadata2, autoload=True, autoload_with=engine)

2022-02-01 01:51:24,978 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("network")
2022-02-01 01:51:24,979 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-01 01:51:24,981 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-02-01 01:51:24,982 INFO sqlalchemy.engine.Engine [raw sql] ('network',)
2022-02-01 01:51:24,983 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("network")
2022-02-01 01:51:24,983 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-01 01:51:24,984 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-02-01 01:51:24,985 INFO sqlalchemy.engine.Engine [raw sql] ('network',)
2022-02-01 01:51:24,987 INFO sqlalchemy.engine.Engine PRAGMA main.index_list("network")
2022-02-01 01:51:24,988 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-01 01:

In [147]:
network_reflected

Table('network', MetaData(), Column('network_id', INTEGER(), table=<network>, primary_key=True, nullable=False), Column('name', VARCHAR(length=100), table=<network>, nullable=False), Column('created_at', DATETIME(), table=<network>, nullable=False), Column('owner_id', INTEGER(), ForeignKey('user.id'), table=<network>), schema=None)

# Inspector

In [148]:
from sqlalchemy import inspect

inspector =  inspect(engine)
inspector

In [149]:
inspector.get_table_names()

2022-02-01 01:51:25,097 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2022-02-01 01:51:25,098 INFO sqlalchemy.engine.Engine [raw sql] ()


['address', 'fancy', 'network', 'user']

In [150]:
inspector.get_columns('network')

2022-02-01 01:51:25,170 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("network")
2022-02-01 01:51:25,174 INFO sqlalchemy.engine.Engine [raw sql] ()


[{'name': 'network_id',
  'type': INTEGER(),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 1},
 {'name': 'name',
  'type': VARCHAR(length=100),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'created_at',
  'type': DATETIME(),
  'nullable': False,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'owner_id',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0}]

In [151]:
inspector.get_foreign_keys('address')

2022-02-01 01:51:25,244 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("address")
2022-02-01 01:51:25,246 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-01 01:51:25,247 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-02-01 01:51:25,248 INFO sqlalchemy.engine.Engine [raw sql] ('address',)


[{'name': None,
  'constrained_columns': ['user_id'],
  'referred_schema': None,
  'referred_table': 'user',
  'referred_columns': ['id'],
  'options': {}}]

In [152]:
inspector.get_pk_constraint('user')

2022-02-01 01:51:25,276 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-02-01 01:51:25,277 INFO sqlalchemy.engine.Engine [raw sql] ('user',)
2022-02-01 01:51:25,278 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("user")
2022-02-01 01:51:25,279 INFO sqlalchemy.engine.Engine [raw sql] ()


{'constrained_columns': ['id'], 'name': None}

# SQL expressions

In [153]:
# To be done................